In [38]:
import curses
import cv2                 # working with, mainly resizing, images
import numpy as np         # dealing with arrays
import os                  # dealing with directories
from tqdm import tqdm 
import pandas as pd
from random import shuffle
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
#Function to Replace Destination File names
import os 
path = os.chdir('C:/Users/Acer/Desktop/jbm/Healthy_test')

# Function to rename multiple files 
def main(): 
    i = 0
      
    for filename in os.listdir(path):
        dst ="Healthy" + "." + str(i) + ".jpeg"
        src = filename
        dst = dst
        i +=1
        os.rename(src, dst) 
        
if __name__ == '__main__': 
    main()              

In [2]:
Healthy ="C:/Users/Acer/Desktop/jbm/Healthy" 
Healthy_test ="C:/Users/Acer/Desktop/jbm/Healthy_test" 

Defects ="C:/Users/Acer/Desktop/jbm/Defect"
Defects_test ="C:/Users/Acer/Desktop/jbm/Defect_test" 

IMG_SIZE = 64
LR = 1e-3

In [3]:
def label_img(img):
    word_label = img.split('.')[0]
    # conversion to one-hot array [non-Defective,defect]
    #                           
    if word_label == 'Healthy': return [1,0]
    #                             [defect,non-Defective]
    elif word_label == 'Defect': return [1,0]

In [4]:
def traimage_read(TRAIN_DIR):
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img,(IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    return training_data

In [5]:
def testimage_read(TEST_DIR):
    test_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        label = label_img(img)
        path = os.path.join(TEST_DIR,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img,(IMG_SIZE,IMG_SIZE))
        test_data.append([np.array(img),np.array(label)])
    return test_data

In [8]:
#For Training Data
Healthy_data=traimage_read(Healthy)

100%|████████████████████████████████████████████████████████████████████████████████| 399/399 [00:30<00:00, 15.83it/s]


In [9]:
Defects_data= testimage_read(Defects)

100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [01:10<00:00,  7.57it/s]


In [12]:
#For Test Data
Healthy_data_test=testimage_read(Healthy_test)

100%|██████████████████████████████████████████████████████████████████████████████████| 95/95 [00:12<00:00,  7.89it/s]


In [13]:
Defect_data_test=testimage_read(Defects_test)

100%|████████████████████████████████████████████████████████████████████████████████| 212/212 [00:21<00:00,  9.88it/s]


In [14]:
np.save('Healthy.npy',Healthy_data)
np.save('Defects.npy',Defects_data)
np.save('Healthy_test.npy',Healthy_data_test)
np.save('Defects_test.npy',Defect_data_test)

In [15]:
del Healthy_data
del Defects_data
del Healthy_data_test
del Defect_data_test

In [16]:
Healthy_data = np.load('Healthy.npy')
Defects_data = np.load('Defects.npy')
Healthy_data_test = np.load('Healthy_test.npy')
Defects_data_test = np.load('Defects_test.npy')

In [24]:
print(Defects_data.shape,Healthy_data.shape)

(600, 2) (399, 2)


In [25]:
train=np.concatenate((Healthy_data,Defects_data))

In [26]:
test=np.concatenate((Healthy_data_test,Defects_data_test))

In [27]:
print(train.shape,test.shape)

(999, 2) (307, 2)


In [28]:
x=np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y=np.array([i[1] for i in train])
x_test=np.array([i[0] for i in test]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_test=np.array([i[1] for i in test])

In [29]:
train_X = x.astype('float32')
test_X = x_test.astype('float32')

In [30]:
train_X = train_X / 255.
test_X = test_X / 255.

In [31]:
from keras.utils import to_categorical
# Change the labels from categorical to one-hot encoding
train_Y_one_hot = to_categorical(y)
test_Y_one_hot = to_categorical(y_test)

# Display the change for category label using one-hot encoding
print('Original label:', y[0])
print('After conversion to one-hot:', train_Y_one_hot[0])

Original label: [1 0]
After conversion to one-hot: [[0. 1.]
 [1. 0.]]


In [32]:
print(x.shape,y.shape)

(999, 64, 64, 1) (999, 2)


In [33]:
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *
from keras.preprocessing.image import ImageDataGenerator,img_to_array, load_img

In [34]:
model = Sequential()
model.add(InputLayer(input_shape=[IMG_SIZE,IMG_SIZE,1]))
model.add(Conv2D(filters=128,kernel_size=5,strides=1,padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=5,padding='same'))
model.add(Dropout(0.8))

model.add(Conv2D(filters=256,kernel_size=5,strides=1,padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=5,padding='same'))
model.add(Dropout(0.8))

model.add(Conv2D(filters=256,kernel_size=5,strides=1,padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=5,padding='same'))
model.add(Dropout(0.8))

model.add(Conv2D(filters=512,kernel_size=5,strides=1,padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=5,padding='same'))

model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(2, activation='softmax'))
optimizer = Adam(lr=1e-3)
model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

In [35]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [36]:
datagen.fit(x)

In [37]:
model.fit_generator(datagen.flow(x,y, batch_size=32),steps_per_epoch=len(x) / 32, epochs=5)

Epoch 1/5
32/31 [==============================] - ETA: 5:51 - loss: 0.8916 - acc: 0.406 - ETA: 2:51 - loss: 0.6224 - acc: 0.640 - ETA: 1:51 - loss: 0.4372 - acc: 0.760 - ETA: 1:21 - loss: 0.3286 - acc: 0.820 - ETA: 1:03 - loss: 0.2630 - acc: 0.856 - ETA: 51s - loss: 0.2192 - acc: 0.880 - ETA: 42s - loss: 0.1879 - acc: 0.89 - ETA: 36s - loss: 0.1644 - acc: 0.91 - ETA: 31s - loss: 0.1461 - acc: 0.92 - ETA: 27s - loss: 0.1315 - acc: 0.92 - ETA: 23s - loss: 0.1195 - acc: 0.93 - ETA: 20s - loss: 0.1096 - acc: 0.94 - ETA: 18s - loss: 0.1012 - acc: 0.94 - ETA: 16s - loss: 0.0939 - acc: 0.94 - ETA: 14s - loss: 0.0877 - acc: 0.95 - ETA: 12s - loss: 0.0822 - acc: 0.95 - ETA: 11s - loss: 0.0774 - acc: 0.95 - ETA: 10s - loss: 0.0731 - acc: 0.96 - ETA: 8s - loss: 0.0692 - acc: 0.9622 - ETA: 7s - loss: 0.0657 - acc: 0.964 - ETA: 6s - loss: 0.0626 - acc: 0.965 - ETA: 6s - loss: 0.0598 - acc: 0.967 - ETA: 5s - loss: 0.0572 - acc: 0.968 - ETA: 4s - loss: 0.0548 - acc: 0.970 - ETA: 3s - loss: 0.0526 - 